### LSTM 문장생성하기
- https://www.kaggle.com/aashita/nyt-comments
- ArticlesApril2018.csv

In [3]:
import pandas as pd

df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324 entries, 0 to 1323
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   articleID         1324 non-null   object
 1   articleWordCount  1324 non-null   int64 
 2   byline            1324 non-null   object
 3   documentType      1324 non-null   object
 4   headline          1324 non-null   object
 5   keywords          1324 non-null   object
 6   multimedia        1324 non-null   int64 
 7   newDesk           1324 non-null   object
 8   printPage         1324 non-null   int64 
 9   pubDate           1324 non-null   object
 10  sectionName       1324 non-null   object
 11  snippet           1324 non-null   object
 12  source            1324 non-null   object
 13  typeOfMaterial    1324 non-null   object
 14  webURL            1324 non-null   object
dtypes: int64(3), object(12)
memory usage: 155.3+ KB


In [5]:
print(df.columns)
print('열의 개수:', len(df.columns))

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')
열의 개수: 15


In [6]:
df['headline'].isnull().values.any() # null 값 있는지 확인

False

In [7]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
len(headline) # 현재 데이터의 개수

1324

In [9]:
headline = [n for n in headline if n != "Unknown"] # Unknown 데이터 제거
len(headline) # 제거 후 데이터의 개수

1214

In [10]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [11]:
from string import punctuation

def repreprocessing(s):
    s = s.encode("utf8").decode("ascii","ignore") # 코드 추가
    return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [12]:
from keras_preprocessing.text import Tokenizer

t = Tokenizer()
t.fit_on_texts(text)

vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [13]:
sequences = list()

for line in text: # 1214개의 데이터에 대해서 데이터를 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 데이터에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11] #11개의 데이터 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [14]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key
    
index_to_word[582]

'offer'

In [15]:
max_len = max(len(i) for i in sequences)
print(max_len)

24


In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [18]:
import numpy as np

sequence = np.array(sequences)

X = sequences[:,:-1]
y = sequences[:,-1]

In [19]:
print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [20]:
print(y[:3])

[ 269  371 1115]


In [21]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes = vocab_size)

In [22]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1)) # y데이터를 분리해서 데이터의 길이 -1
model.add(LSTM(128))
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 200)

Epoch 1/200
244/244 [==============================] - 5s 16ms/step - loss: 7.6293 - accuracy: 0.0314
Epoch 2/200
244/244 [==============================] - 4s 16ms/step - loss: 7.1105 - accuracy: 0.0291
Epoch 3/200
244/244 [==============================] - 4s 16ms/step - loss: 6.9663 - accuracy: 0.0352
Epoch 4/200
244/244 [==============================] - 4s 17ms/step - loss: 6.8226 - accuracy: 0.0388
Epoch 5/200
244/244 [==============================] - 4s 17ms/step - loss: 6.6588 - accuracy: 0.0465
Epoch 6/200
244/244 [==============================] - 4s 17ms/step - loss: 6.4738 - accuracy: 0.0486
Epoch 7/200
244/244 [==============================] - 4s 18ms/step - loss: 6.2769 - accuracy: 0.0537
Epoch 8/200
244/244 [==============================] - 5s 20ms/step - loss: 6.0730 - accuracy: 0.0588 0s - loss: 6
Epoch 9/200
244/244 [==============================] - 5s 22ms/step - loss: 5.8750 - accuracy: 0.0666
Epoch 10/200
244/244 [==============================] - 5s 20ms/step 

244/244 [==============================] - 6s 24ms/step - loss: 1.0259 - accuracy: 0.7942
Epoch 80/200
244/244 [==============================] - 6s 24ms/step - loss: 1.0027 - accuracy: 0.7993 0s - loss: 1.0054 - accuracy
Epoch 81/200
244/244 [==============================] - 6s 25ms/step - loss: 0.9804 - accuracy: 0.8055
Epoch 82/200
244/244 [==============================] - 6s 24ms/step - loss: 0.9606 - accuracy: 0.8082
Epoch 83/200
244/244 [==============================] - 6s 23ms/step - loss: 0.9394 - accuracy: 0.8129
Epoch 84/200
244/244 [==============================] - 6s 23ms/step - loss: 0.9183 - accuracy: 0.8174
Epoch 85/200
244/244 [==============================] - 5s 21ms/step - loss: 0.8968 - accuracy: 0.8197
Epoch 86/200
244/244 [==============================] - 5s 21ms/step - loss: 0.8774 - accuracy: 0.8257
Epoch 87/200
244/244 [==============================] - 5s 22ms/step - loss: 0.8582 - accuracy: 0.8270
Epoch 88/200
244/244 [==============================] - 6

244/244 [==============================] - 5s 21ms/step - loss: 0.2967 - accuracy: 0.9168
Epoch 158/200
244/244 [==============================] - 5s 21ms/step - loss: 0.2952 - accuracy: 0.9166
Epoch 159/200
244/244 [==============================] - 5s 20ms/step - loss: 0.2940 - accuracy: 0.9163
Epoch 160/200
244/244 [==============================] - 5s 22ms/step - loss: 0.2900 - accuracy: 0.9167
Epoch 161/200
244/244 [==============================] - 5s 22ms/step - loss: 0.2875 - accuracy: 0.9170
Epoch 162/200
244/244 [==============================] - 6s 23ms/step - loss: 0.2858 - accuracy: 0.9175
Epoch 163/200
244/244 [==============================] - 6s 23ms/step - loss: 0.2845 - accuracy: 0.9163
Epoch 164/200
244/244 [==============================] - 6s 24ms/step - loss: 0.2854 - accuracy: 0.9166
Epoch 165/200
244/244 [==============================] - 6s 25ms/step - loss: 0.2849 - accuracy: 0.9167
Epoch 166/200
244/244 [==============================] - 6s 26ms/step - loss: 

In [28]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen = 23, padding = 'pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose = 0)
        
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장
        for word, index in t.word_index.items():
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    
    # for 문이므로 이 행동을 다시 반복
    
    sentence = init_word + sentence
    return sentence

In [29]:
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 i에 대해서 10개의 단어를 추가 생성

i disapprove of school vouchers can i still apply for them


In [27]:
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 how에 대해서 10개의 단어를 추가 생성

how to make a crossword puzzle tarot to furor do so
